# Processamento de linguagem Natural
### Um conjunto de dados para Processamento de Linguagem Natural (PLN) é uma coleção de textos ou amostras de linguagem utilizada para treinar, validar e testar modelos de aprendizado de máquina voltados para o processamento de texto. Características desejáveis incluem representatividade, diversidade linguística, tamanho adequado, equilíbrio de classes, anotações de qualidade, variedade de tarefas, atualização regular e considerações de privacidade e ética. Esses conjuntos de dados são essenciais para desenvolver modelos de PLN eficazes e robustos.

### Abaixo temos um exemplo em código usando a biblioteca nltk (Natural Language Toolkit) 

In [1]:
import nltk
from nltk.corpus import brown

# Baixar o corpus 'brown' se ainda não estiver baixado
nltk.download('brown')

# Carregar o corpus 'brown'
corpus = brown.sents(categories='news')  # Vamos usar apenas a categoria 'news' para exemplo

# Exibir alguns exemplos de frases do corpus
for i in range(5):
    print('Exemplo', i+1, ':', ' '.join(corpus[i]))



Exemplo 1 : The Fulton County Grand Jury said Friday an investigation of Atlanta's recent primary election produced `` no evidence '' that any irregularities took place .
Exemplo 2 : The jury further said in term-end presentments that the City Executive Committee , which had over-all charge of the election , `` deserves the praise and thanks of the City of Atlanta '' for the manner in which the election was conducted .
Exemplo 3 : The September-October term jury had been charged by Fulton Superior Court Judge Durwood Pye to investigate reports of possible `` irregularities '' in the hard-fought primary which was won by Mayor-nominate Ivan Allen Jr. .
Exemplo 4 : `` Only a relative handful of such reports was received '' , the jury said , `` considering the widespread interest in the election , the number of voters and the size of this city '' .
Exemplo 5 : The jury said it did find that many of Georgia's registration and election laws `` are outmoded or inadequate and often ambiguous '

[nltk_data] Downloading package brown to /home/gabriel/nltk_data...
[nltk_data]   Package brown is already up-to-date!


# char-RNN (Character-Level Recurrent Neural Network)

### Char-RNN (Character-Level Recurrent Neural Network) é um tipo de rede neural recorrente projetado para processar e gerar texto com base em caracteres individuais. Em contraste com abordagens de processamento de linguagem natural que operam em níveis de palavra, o char-RNN trabalha diretamente com sequências de caracteres. 

In [3]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import nltk
from nltk.corpus import brown

# Baixar o corpus 'brown' se ainda não estiver baixado
nltk.download('brown')

# Carregar o corpus 'brown'
corpus = brown.sents(categories='news')
text = ' '.join([' '.join(sentence) for sentence in corpus])

# Criar um conjunto de dados de treinamento
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
max_len = 40  # Comprimento máximo da sequência de entrada
step = 3  # Passo para criar sequências de entrada

sentences = []
next_chars = []
for i in range(0, len(text) - max_len, step):
    sentences.append(text[i:i + max_len])
    next_chars.append(text[i + max_len])

# Vetorização dos dados
x = np.zeros((len(sentences), max_len, len(chars)), dtype=np.int8)
y = np.zeros((len(sentences), len(chars)), dtype=np.int8)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

# Definir o modelo char-RNN
model = Sequential()
model.add(LSTM(128, input_shape=(max_len, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

# Compilar o modelo
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Treinar o modelo
model.fit(x, y, batch_size=128, epochs=5)

# Função para gerar texto usando o modelo treinado
def generate_text(seed_text, model, max_len, char_indices, indices_char):
    generated_text = seed_text
    for _ in range(400):
        x_pred = np.zeros((1, max_len, len(chars)), dtype=np.int8)
        for t, char in enumerate(seed_text):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = np.argmax(preds)
        next_char = indices_char[next_index]

        seed_text = seed_text[1:] + next_char
        generated_text += next_char

    return generated_text

# Gerar texto usando o modelo treinado
seed_text = "The weather is"
generated_text = generate_text(seed_text, model, max_len, char_indices, indices_char)
print(generated_text)


[nltk_data] Downloading package brown to /home/gabriel/nltk_data...
[nltk_data]   Package brown is already up-to-date!


Epoch 1/5


2023-11-24 21:37:27.257679: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 564832320 exceeds 10% of free system memory.


1415/1415 [==============================] - 104s 72ms/step - loss: 2.6546
Epoch 2/5
1415/1415 [==============================] - 111s 78ms/step - loss: 2.2774
Epoch 3/5
1415/1415 [==============================] - 111s 79ms/step - loss: 2.1606
Epoch 4/5
1415/1415 [==============================] - 111s 79ms/step - loss: 2.0886
Epoch 5/5
1415/1415 [==============================] - 112s 79ms/step - loss: 2.0328
The weather is                                                                                                                                                                                                                                                                                                                                                                                                                


# Modelo de Atenção

### Um modelo de Atenção é uma arquitetura de rede neural projetada para tarefas envolvendo sequências de dados, como tradução automática e processamento de linguagem natural. Diferentemente de abordagens tradicionais, o modelo de Atenção permite que a rede se concentre em partes específicas da entrada em cada passo, ponderando a importância de diferentes elementos. Isso melhora o desempenho em tarefas que envolvem sequências longas, e o modelo pode aprender a destacar informações relevantes dinamicamente. 

In [4]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Attention
import nltk
from nltk.corpus import brown

# Baixar o corpus 'brown' se ainda não estiver baixado
nltk.download('brown')

# Carregar o corpus 'brown'
corpus = brown.sents(categories='news')
text = ' '.join([' '.join(sentence) for sentence in corpus])

# Criar um conjunto de dados de treinamento
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
max_len = 40  # Comprimento máximo da sequência de entrada
step = 3  # Passo para criar sequências de entrada

sentences = []
next_chars = []
for i in range(0, len(text) - max_len, step):
    sentences.append(text[i:i + max_len])
    next_chars.append(text[i + max_len])

# Vetorização dos dados
x = np.zeros((len(sentences), max_len, len(chars)), dtype=np.int8)
y = np.zeros((len(sentences), len(chars)), dtype=np.int8)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

# Definir o modelo char-RNN com atenção
input_seq = Input(shape=(max_len, len(chars)))
lstm_out = LSTM(128, return_sequences=True)(input_seq)
attention = Attention()([lstm_out, lstm_out])
context = attention
merged = Model(inputs=input_seq, outputs=context)

model = Sequential()
model.add(merged)
model.add(LSTM(128))
model.add(Dense(len(chars), activation='softmax'))

# Compilar o modelo
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Treinar o modelo
model.fit(x, y, batch_size=128, epochs=5)


[nltk_data] Downloading package brown to /home/gabriel/nltk_data...
[nltk_data]   Package brown is already up-to-date!


Epoch 1/5


2023-11-24 21:53:05.320377: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 564832320 exceeds 10% of free system memory.
W0000 00:00:1700873588.406782   28814 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" vendor: "AuthenticAMD" model: "248" frequency: 2096 num_cores: 8 environment { key: "cpu_instruction_set" value: "AVX SSE, SSE2, SSE3, SSSE3, SSE4.1, SSE4.2" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 32768 l2_cache_size: 524288 l3_cache_size: 4194304 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


1415/1415 [==============================] - 250s 175ms/step - loss: 3.1183
Epoch 2/5
1415/1415 [==============================] - 267s 189ms/step - loss: 2.5457
Epoch 3/5
1415/1415 [==============================] - 268s 189ms/step - loss: 2.0716
Epoch 4/5
1415/1415 [==============================] - 267s 189ms/step - loss: 1.9318
Epoch 5/5
1415/1415 [==============================] - 268s 189ms/step - loss: 1.8362
